In [ ]:
# import Keras
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional. import Convolutional2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Cropping2D

import cv2
import os
import csv

# open CSV file containing paths to simulator images
data_paths = []
with open('driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        data_paths.append(line)

# populate containers to hold images and corresponding steering angles
images = []
measurements = []

for line in data_paths:
    
    # read in center, left, and right camera images from simulator
    for i in range(3):
        source_path = line[i]
        tokens = source_path.split('/')
        filename = tokens[-1]
        local_path = "IMG/" + filename

        # load the image in the BGR format and convert to YUV
        image = cv2.cvtColor(cv2.imread(local_path, CV_LOAD_IMAGE_COLOR), cv2.COLOR_BGR2YUV)
        
        images.append(image)
    
    # correction to apply to left and right camera images
    correction = 0.015
    
    # get the steering angle recorded during simulator driving
    measurement = float(line[3])
    
    # don't add correction to center camera image
    measurements.append(measurement)
    
    # steer more to the right for left camera images
    measurements.append(measurement + correction)
    
    # steer more to the left for right camera images
    measurements.append(measurement - correction)
    
# flip the images in order to simulate driving the reverse direction on the track
# note: this process can be combined with the loop above, but its
# written here for clarity of the process.
# 2nd note: more image preprocessing can be done here, like image warping, if desired.
augmented_images = []
augmented_measurements = []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    flipped_image = cv2.flip(image, 1)
    flipped_measurement = float(measurement) * -1.0
    augmented_images.append(flipped_image)
    augmented_measurements.append(flipped_measurement)
    
# cast the training data set into numpy arrays
x_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

# get model
model = Sequential()

# The inputs are YUV, having values between 0 and 255.
# Normalize the inputs between a range of 0 and 1 by dividing by 255.
# Subtracting by 0.5 will shift mean from 0.5 to 0.
# Normalizing the data helps the network learn more efficiently.
# The input shape of the image is 160x320 with 3 color channels for YUV
model.add(Lambda(lambda X: x / 255.0 - 0.5, input_shape=(160, 320, 3)))

# Crop the image to only show relevant pixels (crop out trees, mountains, and the hood of the car)
# that are critical to the network. Just the road itself.
model.add(Cropping2D(cropping=((70,25), (0,0))))

# The following network architecture was taken from the paper,
# "End to End Learning for Self-Driving Cars" by several engineers
# at Nvidia. The paper can be accessed here:
# http://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf

model.add(Convolution2D(24,5,5,subsample=(2,2), activation="relu"))
model.add(Convolution2D(36,5,5,subsample=(2,2), activation="relu"))
model.add(Convolution2D(48,5,5,subsample=(2,2), activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Flatten())
model.add(Dense(1164))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Dense(1))

# implement the Adam optimizer
adamOpt = keras.optimizers.Adam(lr=0.001, beta_l=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='mse', optimizer=adamOpt)

history = model.fit(x_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=10)

model.save("model.h5")





























